# Create evaluation dataset for Redbox RAG chat  <a class="anchor" id="title"></a>

**Before running this notebook**

Set the version of the evaluation dataset you are creating **[HERE](#setversion)**

## Table of Contents <a class="anchor" id="toc"></a>
* [Set version of the evaluation dataset you are creating](#setversion)
* [Overview](#overview)
* [Notebook Setup](#setup)
* [Generate Evaluation Dataset](#ragas)
* [Save Evaluation Dataset](#six-section)
* [Troubleshooting](#troubleshooting)
* [#TODO - Next Steps](#todo)

## Overview <a class="anchor" id="overview"></a>

There is a troubleshooting section at the end of this notebook [Troubleshooting](#troubleshooting)

[Back to top](#title)

## Notebook Setup <a id="setup"></a>

In [3]:
from tqdm.auto import tqdm
import pandas as pd
from typing import Optional, List, Tuple
import json

pd.set_option("display.max_colwidth", None)

[Back to top](#title)

------

## Set version of the evaulation dataset you are creating <a class="anchor" id="setversion"></a>

**Before running this notebook**

Set the version of the evaluation dataset you are creating

In [14]:
version = "0.1.0"

**It is really important to version the evaluations we are doing**, including the input data used to generate evaluation datasets

#### Create a data directory this this version of evaluation data

In [ ]:
from pathlib import Path
dir_name = f"evaluation_data_v{version}"
dir_path = Path(f"./data/{dir_name}")

if not dir_path.exists():
    dir_path.mkdir(parents=True)

**Now copy all the files you want to use to generate your evaluation dataset into the new directory created above**

[Back to top](#title)

---------------

## Synthetically generate evaluation dataset <a class="anchor" id="ragas"></a>

RAGAS generating a synthetic test set detailed [HERE](https://docs.ragas.io/en/stable/getstarted/testset_generation.html). Perhaps not as SOTA as DeepEval (validate!), but it creates `input` AND `expected_output` for us. 

So we are not generating input questions based on our chunking strategy, however, we are using the same files

In [15]:
# Takes about 4 minutes for 4 docs. Consider Langchain `unstructured`
from langchain.document_loaders import DirectoryLoader
# loader = DirectoryLoader("./data/evaluation_files")
loader = DirectoryLoader(f"./data/{dir_name}")
documents = loader.load()

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


#### Save Langchain documents for future use

In [16]:
import typing as t
import jsonlines
from langchain.schema import Document


def save_docs_to_jsonl(documents: t.Iterable[Document], file_path: str) -> None:
    with jsonlines.open(file_path, mode="w") as writer:
        for doc in documents:
            writer.write(doc.dict())


def load_docs_from_jsonl(file_path) -> t.Iterable[Document]:
    documents = []
    with jsonlines.open(file_path, mode="r") as reader:
        for doc in reader:
            documents.append(Document(**doc))
    return documents

In [17]:
#TODO: Use the functions above!
save_docs_to_jsonl(documents, f"./data/synthetic_data/langchain_documents_for_ragas_v{version}.jsonl")

-----------

In [18]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# generator with openai models
generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
critic_llm = ChatOpenAI(model="gpt-4")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

## TODO: Next steps  <a class="anchor" id="todo"></a>

In [22]:
#TODO: Add code to handle rate limits in the generator - partiuclarly for the critic using GPT-4: https://github.com/openai/openai-cookbook/blob/main/examples/How_to_handle_rate_limits.ipynb

In [ ]:
#TODO: Replace the `generate_with_langchain_docs` method with something utilising Redbox chunking strategy

In [25]:
#TODO: Investigate why RAGAS sometimes creates NaN values!

In [ ]:
#TODO: Can we add cost estimate to RAGAS synthetic test generation?

**CHANGE `test_size` to generate more evaluation data** (in cell below)

In [ ]:
# generate testset
testset = generator.generate_with_langchain_docs(documents, test_size=10, distributions={simple: 0.4, reasoning: 0.3, multi_context: 0.3})

#### Save RAGAS generated testset as pickle

In [10]:
import pickle

with open(f'./data/synthetic_data/ragas_testset_v{version}.pkl', 'wb') as f:
    pickle.dump(testset, f)

Export to Pandas

In [ ]:
## To view dataframe in notebook (very text heavy!)
# testset.to_pandas()

#### Convert dataframe into a DeepEval compatible CSV & save

In [21]:
testset_df = testset.to_pandas()

# Rename the columns
new_column_names = {
    'question': 'input',
    'contexts': 'context',
    'ground_truth': 'expected_output',
    # Add more column names here
}

testset_df_renamed = testset_df.rename(columns=new_column_names)

#  DeepEval dataset format requires an 'actual_output' column
testset_df_renamed['actual_output'] = ''
testset_df_renamed = testset_df_renamed.drop(['evolution_type', 'metadata', 'episode_done'], axis=1)

# Convert all columns to string - otherwise DeepEval will throw an Pydantic validation error
testset_df_renamed = testset_df_renamed.astype(str)

# save as CSV
testset_df_renamed.to_csv(f'./data/synthetic_data/ragas_synthetic_data_v{version}.csv', index=False)

In [26]:
testset_df_renamed.head()

input  \
0  What were the findings regarding the prevalence of behavioural disorders among EBCI children who received UBI-like payments?   
1                                                         How is occupational exposure to AI categorized based on skill levels?   
2                                        What are the mental health effects of conditional versus unconditional cash transfers?   
3               What were the effects of cash payments on recipients in the B-MINCOME study and the Ontario Basic Income Pilot?   
4                                                          Why is coordinated action necessary in addressing frontier AI risks?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

#### Convert dataframe into a DeepEval compatible JSON

In [ ]:
# Convert the DataFrame to a JSON object
ragas_synthetic_data_json = testset_df_renamed.to_json(orient='records')

data = json.loads(ragas_synthetic_data_json)

# Convert the Python object back to a JSON string, with indentation for prettifying
pretty_json = json.dumps(data, indent=4)

# Define the path to the output file
output_file_path = f'./data/synthetic_data/ragas_synthetic_data_v{version}.json'


# Save the JSON object to a file
with open(output_file_path, 'w') as f:
    json.dump(pretty_json, f)

In [ ]:
data = json.loads(ragas_synthetic_data_json)

# Convert the Python object back to a JSON string, with indentation for prettifying
pretty_json = json.dumps(data, indent=4)

print(pretty_json)

[Back to top](#title)

-----------------------

## Trouble shooting <a class="anchor" id="troubleshooting"></a>

#### Langchain DirectoryLoader Error

If you run into a poppler path error and poppler is installed and can be access from your virtual environment (by running `pdfinfo -v`), then close notebook and restart the Jupyter server from the terminal where the path is correctly set (by running `code notebooks/evaluation/evaluation_dataset_generation.ipynb`) 

#### RAGAS synthetically generated evaluation data

We have found some rows of synthetically generated evaluation data from using the RAGAS framework, includes some NaN and/or not str type, which results in an error for DeepEval metrics, as these data fail Pydantic validation.

To avoid this, ensure you turn RAGAS synthetically generated evaluation data to type str and remove rows of data with NaN

#### DeepEval framework

At the moment, I can only load from CSV into DeepEval test cases, so there may be something wrong with the JSON created above. #TODO: Debug

[Back to top](#title)

-------